In [1]:
link = 'D:/users/Marko/downloads/mirna/'

# Imports

In [2]:
%load_ext tensorboard

In [3]:
import sys
#sys.path.insert(0,'/content/drive/MyDrive/Marko/master')
sys.path.insert(0, link)
import numpy as np
import matplotlib.pyplot as plt

#import tensorflow as tf

import torch
import torch.optim as optim
import torch.nn as nn
import torch.distributions as dist

from torch.nn import functional as F
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder

from tqdm import tqdm
from tqdm import trange

import datetime


writer = SummaryWriter(f"{link}/saved_models/new/NVAE5/tensorboard")

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
DEVICE

device(type='cuda')

# Model Classes

In [6]:
class diva_args:

    def __init__(self, z_dim=128, d_dim=45, x_dim=7500, y_dim=2,
                 beta=10, rec_alpha = 1, rec_beta = 1, 
                 rec_gamma = 1, warmup = 1, prewarmup = 1):

        self.z_dim = z_dim
        self.d_dim = d_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        
        self.beta = beta
        self.rec_alpha = rec_alpha
        self.rec_beta = rec_beta
        self.rec_gamma = rec_gamma
        self.warmup = warmup
        self.prewarmup = prewarmup


## Dataset Class

In [7]:
class MicroRNADataset(Dataset):

    def __init__(self, ds='train', create_encodings=False, use_subset=False):
        
        # loading images
        self.images_org = np.load(f'{link}/data/modmirbase_{ds}_images.npz')['arr_0']/255
        
        # loading labels
        print('Loading Labels! (~10s)')     
        ohe = OneHotEncoder(categories='auto', sparse=False)
        labels = np.load(f'{link}/data/modmirbase_{ds}_labels.npz')['arr_0']
        self.labels = ohe.fit_transform(labels)
        
        # loading encoded images
        print("loading encodings")
        if create_encodings:
            x_len, x_bar, x_col, images = self.get_encoded_values(self.images_org, ds)
        else:
            x_len = np.load(f'{link}/data/modmirbase_{ds}_images_len_new.npz')
            x_bar = np.load(f'{link}/data/modmirbase_{ds}_images_bar_new.npz')
            x_col = np.load(f'{link}/data/modmirbase_{ds}_images_col_new.npz')
            images= np.load(f'{link}/data/modmirbase_{ds}_images_cat_new.npz')
        
        self.x_len = x_len
        self.x_bar = x_bar
        self.x_col = x_col
        self.images= images
        
        
        self.mountain = np.load(f'{link}/data/modmirbase_{ds}_mountain.npy')
        
        
        # loading names
        print('Loading Names! (~5s)')
        names =  np.load(f'{link}/data/modmirbase_{ds}_names.npz')['arr_0']
        names = [i.decode('utf-8') for i in names]
        self.species = ['mmu', 'prd', 'hsa', 'ptr', 'efu', 'cbn', 'gma', 'pma',
                        'cel', 'gga', 'ipu', 'ptc', 'mdo', 'cgr', 'bta', 'cin', 
                        'ppy', 'ssc', 'ath', 'cfa', 'osa', 'mtr', 'gra', 'mml',
                        'stu', 'bdi', 'rno', 'oan', 'dre', 'aca', 'eca', 'chi',
                        'bmo', 'ggo', 'aly', 'dps', 'mdm', 'ame', 'ppc', 'ssa',
                        'ppt', 'tca', 'dme', 'sbi']
        # assigning a species label to each observation from species
        # with more than 200 observations from past research
        self.names = []
        for i in names:
            append = False
            for j in self.species:
                if j in i.lower():
                    self.names.append(j)
                    append = True
                    break
            if not append:
                if 'random' in i.lower() or i.isdigit():
                    self.names.append('hsa')
                else:
                    self.names.append('notfound')
        
        # performing one hot encoding
        ohe = OneHotEncoder(categories='auto', sparse=False)
        
       
        
        self.names_ohe = ohe.fit_transform(np.array(self.names).reshape(-1,1))
          
        if use_subset:    
            idxes = [i == 'hsa' and np.random.choice([True, False]) for i in self.names]
            self.names_ohe = self.names_ohe[idxes]
            self.labels = self.labels[idxes]
            self.images = self.images[idxes]
            self.images_org = self.images_org[idxes]
            self.x_len = self.x_len[idxes]
            self.x_col = self.x_col[idxes]
            self.x_bar = self.x_bar[idxes]
            self.mountain = self.mountain[idxes]
    
    def __len__(self):
        return(self.images.shape[0])

    def __getitem__(self, idx):
        d = self.names_ohe[idx]
        y = self.labels[idx]
        x = self.images[idx]
        x = np.transpose(x, (2,0,1))
        x_len = self.x_len[idx]
        x_col = self.x_col[idx]
        x_bar = self.x_bar[idx]
        mount = self.mountain[idx]
        x_org = np.transpose(self.images_org[idx], (2,0,1))
        return (x, y, d, x_len, x_col, x_bar, mount, x_org)


    def get_encoded_values(self, x, ds):
        """
        given an image or batch of images
        returns length of strand, length of bars and colors of bars
        """
        n = x.shape[0]
        x = np.transpose(x, (0,3,1,2))
        out_len = np.zeros((n), dtype=np.uint8)
        out_col = np.zeros((n,26,100), dtype=np.uint8)
        out_bar = np.zeros((n,2,100), dtype=np.uint8)
        out_x = np.zeros((n,25,100,5))

        for i in range(n):
            if i % 100 == 0:
                print(f'at {i} out of {n}')
            rna_len = 0
            broke = False
            for j in range(100):
                if (x[i,:,12,j] == np.array([1,1,1])).all():
                    out_len[i] = rna_len
                    broke = True
                    out_col[i,25,j] = 1
                else:
                    rna_len += 1
                    # check color of bars
                    out_col[i, self.get_color(x,i,j),j] = 1 
                    #out_col[i, self.get_color(x[i,:,13,j]), 1, j] = 1
                    # check length of bars
                    len1 = 0
                    # loop until white pixel
                    while not (x[i,:,12-len1,j] == np.array([1.,1.,1.])).all():
                        len1 += 1
                        if 13-len1 == 0:
                            break
                    out_bar[i, 0, j] = len1
                    tc = int(self._get_color(x[i,:,12,j]))
                    out_x[i,13-len1:13,j,tc]=1
                    

                    len2 = 0
                    while not (x[i,:,13+len2,j] == np.array([1.,1.,1.])).all():
                        len2 += 1
                        if 13+len2 == 25:
                            break
                    out_bar[i, 1, j] = len2
                    bc = int(self._get_color(x[i,:,13,j]))
                    out_x[i,13:13+len2,j,bc]=1
            if not broke:
                out_len[i] = rna_len


        with open(f'{link}/data/modmirbase_{ds}_images_len_new.npz', 'wb') as f:
            np.save(f, out_len)
        with open(f'{link}/data/modmirbase_{ds}_images_col_new.npz', 'wb') as f:
            np.save(f, out_col)
        with open(f'{link}/data/modmirbase_{ds}_images_bar_new.npz', 'wb') as f:
            np.save(f, out_bar)
        with open(f'{link}/data/modmirbase_{ds}_images_cat_new.npz', 'wb') as f:
            np.save(f, out_x)
        

        return out_len, out_bar, out_col, out_x

    def get_color(self, x, i, j):
        
        col = self._get_color(x[i,:,12,j])+self._get_color(x[i,:,13,j])
        if col == '00':
            return 0
        if col == '01':
            return 1
        if col == '02':
            return 2
        if col == '03':
            return 3
        if col == '04':
            return 4
        if col == '10':
            return 5
        if col == '11':
            return 6
        if col == '12':
            return 7
        if col == '13':
            return 8
        if col == '14':
            return 9
        if col == '20':
            return 10
        if col == '21':
            return 11
        if col == '22':
            return 12
        if col == '23':
            return 13
        if col == '24':
            return 14
        if col == '30':
            return 15
        if col == '31':
            return 16
        if col == '32':
            return 17
        if col == '33':
            return 18
        if col == '34':
            return 19
        if col == '40':
            return 20
        if col == '41':
            return 21
        if col == '42':
            return 22
        if col == '43':
            return 23
        if col == '44':
            return 24
        
        
    
    def _get_color(self, pixel):
        """
        returns the encoded value for a pixel
        """
        if (pixel == np.array([0,0,0])).all():  
            return "0" # black
        elif (pixel == np.array([1,0,0])).all():  
            return "1" # red
        elif (pixel == np.array([0,0,1])).all():  
            return "2" # blue
        elif (pixel == np.array([0,1,0])).all():  
            return "3" # green
        elif (pixel == np.array([1,1,0])).all():  
            return "4" # yellow
        else:
            print("Something wrong!")


## Decoder classes

In [8]:
# Decoders
class px(nn.Module):
    def __init__(self, d_dim, x_dim, y_dim, z_dim, dim1=256, dim2=512):
        super(px, self).__init__()

        self.fc = nn.Sequential(nn.Linear(z_dim, dim1, bias=False),  
                                 nn.ReLU(),
                                nn.Linear(dim1, dim2),
                                nn.ReLU())
        
        # Predicting length and color of each bar
        
        self.color = nn.Sequential(nn.Linear(dim2, 2600),
                                   )
        
        
        self.length_bar = nn.Sequential(nn.Linear(dim2,200), nn.Softplus())
        
        
    def forward(self, z):
        
        h = self.fc(z)
        
        
        
        len_bar = self.length_bar(h).reshape(-1,2,100)
        len_bar_sc = nn.Parameter(torch.tensor([1.])).to(DEVICE)
        
        
        col = self.color(h).reshape(-1,26,100)
        col_bar = nn.Softmax(dim=1)(col)
        
        return len_bar, len_bar_sc, col_bar

    def reconstruct_image(self, len_bar, var_bar ,col_bar, sample=False):
        """
        reconstructs RNA image given output from decoder
        even indexes of len_bar and col_bar   -> top
        uneven indexes of len_bar and col_bar -> bottom
        function does not support sampling yet
        color reconstructions: 0: black
                               1: red
                               2: blue
                               3: green
                               4: yellow
        """
        color_dict = {
                  0: np.array([0,0,0]), # black
                  1: np.array([1,0,0]), # red
                  3: np.array([0,1,0]), # green
                  2: np.array([0,0,1]), # blue
                  4: np.array([1,1,0]),  # yellow
                  5: np.array([1,1,1])  # white
                  }
    
        _color_dict =  {0: (0,0),
                        1: (0,1),
                        2: (0,2),
                        3: (0,3),
                        4: (0,4),
                        5: (1,0),
                        6: (1,1),
                        7: (1,2),
                        8: (1,3),
                        9: (1,4),
                        10: (1,0),
                        11: (2,1),
                        12: (2,2),
                        13: (2,3),
                        14: (2,4),
                        15: (2,0),
                        16: (3,1),
                        17: (3,2),
                        18: (3,3),
                        19: (3,4),
                        20: (3,0),
                        21: (4,1),
                        22: (4,2),
                        23: (4,3),
                        24: (4,4),
                        25: (5,5)
                        }       
        len_bar = len_bar.cpu().numpy()
        var_bar = var_bar.cpu().numpy()
        col_bar = col_bar.cpu().numpy()
        n = len_bar.shape[0]
        output = np.ones((n,25,100,3))

        for i in range(n):
            limit = 100
            for j in range(limit):
                if sample:
                    _len_bar_1 = int(np.round(np.random.normal(loc=len_bar[i,0,j], scale=var_bar[i,0,j])))
                    _len_bar_2 = int(np.round(np.random.normal(loc=len_bar[i,1,j], scale=var_bar[i,1,j])))
                    _col_bar_1 = np.random.choice(np.arange(5), p = col_bar[i, :, 2*j])
                    _col_bar_2 = np.random.choice(np.arange(5), p = col_bar[i,:, 2*j+1])
                else:
                    _len_bar_1 = int(np.round(len_bar[i,0,j])) 
                    _len_bar_2 = int(np.round(len_bar[i,1,j]))
                    _col_bar_1, _col_bar_2 = _color_dict[np.argmax(col_bar[i,:,j])]
                    
                h1 = max(0,13-_len_bar_1)
                # paint upper bar
                output[i, h1:13, j] = color_dict[_col_bar_1]
                h2 = min(25,13+_len_bar_2)
                # paint lower bar
                output[i, 13:h2, j] = color_dict[_col_bar_2]
        
        
        return output

In [9]:
int(np.round(3.7, 0))
int(3.7)

3

In [10]:
# pzy_ = pzy(45, 7500, 2, 32,32,32)
# summary(pzy_, (1,2))
# pzy_ = px(45, 7500, 2, 32,32,32)
# summary(pzy_, [(1,32),(1,32),(1,32)])

## Endcoder Classes

In [11]:
#pzy_.reconstruct_image(torch.zeros((1,100)), torch.zeros((1,13,200)), torch.zeros(1,5,200)).shape

In [36]:
class qz(nn.Module):
    def __init__(self, d_dim, x_dim, y_dim, z_dim, h_dim=1152):
        super(qz, self).__init__()
        self.h_dim = h_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(5, 32, kernel_size=9, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=9, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 96, kernel_size=3, stride=1, padding = 'same'),
            nn.ReLU(), 
            nn.Conv2d(96, 128, kernel_size=3, stride=1, padding = 'same'),
            nn.ReLU(), 
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 192, kernel_size=3, stride=1, padding = 'same'),
            nn.ReLU(), 
            nn.Conv2d(192, 192, kernel_size=3, stride=1, padding = 'same'),
            nn.ReLU(), 
            nn.MaxPool2d(2, 2),
        )
        
        self.col = nn.Sequential(nn.Linear(2600, self.h_dim), nn.ReLU())
        
        self.fc11 = nn.Sequential(nn.Linear(2*self.h_dim, z_dim))
        self.fc12 = nn.Sequential(nn.Linear(2*self.h_dim, z_dim), nn.Softplus())


    def forward(self, x, x_col):
        x_col = x_col.reshape(-1,2600)
        c = self.col(x_col)
        h = self.encoder(x)
        h = h.view(-1, self.h_dim)
        ch = torch.cat([c,h],1)
        z_loc = self.fc11(ch)
        z_scale = self.fc12(ch) + 1e-7

        return z_loc, z_scale




In [37]:
enc = qz(128,10,10,1024)
summary(enc, [(1,5,25,100),(1,100,26)])

Layer (type:depth-idx)                   Output Shape              Param #
qz                                       --                        --
├─Sequential: 1-1                        [1, 1152]                 --
│    └─Linear: 2-1                       [1, 1152]                 2,996,352
│    └─ReLU: 2-2                         [1, 1152]                 --
├─Sequential: 1-2                        [1, 192, 1, 6]            --
│    └─Conv2d: 2-3                       [1, 32, 25, 100]          12,992
│    └─ReLU: 2-4                         [1, 32, 25, 100]          --
│    └─Conv2d: 2-5                       [1, 32, 25, 100]          82,976
│    └─ReLU: 2-6                         [1, 32, 25, 100]          --
│    └─MaxPool2d: 2-7                    [1, 32, 12, 50]           --
│    └─Conv2d: 2-8                       [1, 64, 12, 50]           18,496
│    └─ReLU: 2-9                         [1, 64, 12, 50]           --
│    └─Conv2d: 2-10                      [1, 64, 12, 50]          

## Full model class

In [38]:
class StampDIVA(nn.Module):
    def __init__(self, args):
        super(StampDIVA, self).__init__()
        self.z_dim = args.z_dim
        self.d_dim = args.d_dim
        self.x_dim = args.x_dim
        self.y_dim = args.y_dim

        self.px = px(self.d_dim, self.x_dim, self.y_dim, self.z_dim)
        
        self.qz = qz(self.d_dim, self.x_dim, self.y_dim, self.z_dim)
        

        self.beta = args.beta
        
        self.rec_alpha = args.rec_alpha
        self.rec_beta = args.rec_beta
        self.rec_gamma = args.rec_gamma

        self.warmup = args.warmup
        self.prewarmup = args.prewarmup

        self.cuda()

    def forward(self, d, x, y, x_col):
        # Encode
        zd_q_loc, zd_q_scale = self.qz(x, x_col)
        
        # Reparameterization trick
        qz = dist.Normal(zd_q_loc, zd_q_scale)
        z_q = qz.rsample()
        
        
        # Decode
        x_bar, x_bar_scale, x_col = self.px(z_q)
        z_p_loc, z_p_scale = torch.zeros(z_q.size()[0], self.z_dim).cuda(),\
                        torch.ones(z_q.size()[0], self.z_dim).cuda()
        pz = dist.Normal(z_p_loc, z_p_scale)

        # Reparameterization trick
        pz = dist.Normal(z_p_loc, z_p_scale)
        
        return x_bar, x_bar_scale, x_col, qz, pz, z_q

    def loss_function(self, d, x, y, out_len, out_bar, out_col):
        
        x_bar, x_bar_scale, x_col, qz, pz, z_q = self.forward(d, x, y, out_col)
       
        mse_bar = (((x_bar - out_bar)**2)).mean(dim=(1,2)).sum()
        
        max_bar = torch.argmax(x_col, dim=1)
        acc_bar = (max_bar==torch.argmax(out_col, dim=1)).float().mean((1)).sum().float()
        
        CE_bar = mse_bar#-log_bar
        CE_col = F.cross_entropy(x_col, out_col, reduction='sum')

        KL_z = torch.sum(pz.log_prob(z_q) - qz.log_prob(z_q))
          
        return self.rec_beta * CE_bar \
                  + self.rec_gamma * CE_col \
                  - self.beta * KL_z, \
                  CE_bar, CE_col, mse_bar, acc_bar

In [39]:
default_args = diva_args(z_dim=256, rec_alpha = 10, rec_beta = 10, rec_gamma = 10, 
                         beta=1, warmup=1, prewarmup=0)
enc = StampDIVA(default_args)
summary(enc,[ (1,1),(1,5,25,100),(1,1),(1,100,26)])

Layer (type:depth-idx)                   Output Shape              Param #
StampDIVA                                --                        --
├─qz: 1-1                                [1, 256]                  --
│    └─Sequential: 2-1                   [1, 1152]                 --
│    │    └─Linear: 3-1                  [1, 1152]                 2,996,352
│    │    └─ReLU: 3-2                    [1, 1152]                 --
│    └─Sequential: 2-2                   [1, 192, 1, 6]            --
│    │    └─Conv2d: 3-3                  [1, 32, 25, 100]          12,992
│    │    └─ReLU: 3-4                    [1, 32, 25, 100]          --
│    │    └─Conv2d: 3-5                  [1, 32, 25, 100]          82,976
│    │    └─ReLU: 3-6                    [1, 32, 25, 100]          --
│    │    └─MaxPool2d: 3-7               [1, 32, 12, 50]           --
│    │    └─Conv2d: 3-8                  [1, 64, 12, 50]           18,496
│    │    └─ReLU: 3-9                    [1, 64, 12, 50]          

# Training the model

## Loading dataset

In [40]:
RNA_dataset = MicroRNADataset(create_encodings=False)

Loading Labels! (~10s)
loading encodings
Loading Names! (~5s)


In [41]:
RNA_dataset_test = MicroRNADataset('test', create_encodings=False)

Loading Labels! (~10s)
loading encodings
Loading Names! (~5s)


In [42]:
len(RNA_dataset)

34721

In [43]:
def train_single_epoch(train_loader, model, optimizer, epoch):
    model.train()
    train_loss = 0
    epoch_bar_loss = 0
    epoch_col_loss = 0
    no_batches = 0
    mse_bar = 0
    acc_bar = 0
    pbar = tqdm(enumerate(train_loader), unit="batch", 
                                     desc=f'Epoch {epoch}')
    for batch_idx, (x, y, d, x_len, x_col, x_bar,_,_) in pbar:
        # To device
        x, y, d , x_len, x_bar, x_col = x.to(DEVICE), y.to(DEVICE), d.to(DEVICE), x_len.to(DEVICE), x_bar.to(DEVICE), x_col.to(DEVICE)

        optimizer.zero_grad()
        loss, bar_loss, col_loss, mse, acc = model.loss_function(d.float(), x.float(), y.float(), x_len.float(), x_bar.float(), x_col.float())
      
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item()/x.shape[0])
        train_loss += loss
        epoch_bar_loss += bar_loss
        epoch_col_loss += col_loss
        mse_bar += mse
        acc_bar += acc
        no_batches += 1

    train_loss /= len(train_loader.dataset)
    epoch_bar_loss /= len(train_loader.dataset)
    epoch_col_loss /= len(train_loader.dataset)
    acc_bar /= len(train_loader.dataset)
    mse_bar /= len(train_loader.dataset)
    
    return train_loss, epoch_bar_loss, epoch_col_loss, mse_bar, acc_bar

In [44]:
def test_single_epoch(test_loader, model, epoch):
    model.eval()
    test_loss = 0
    epoch_bar_loss = 0
    epoch_col_loss = 0
    mse_bar = 0
    acc_bar = 0        
    with torch.no_grad():
        for batch_idx, (x,y,d,x_len,x_col,x_bar,_,_) in enumerate(test_loader):
            x, y, d, x_len, x_bar, x_col = x.to(DEVICE), y.to(DEVICE), d.to(DEVICE), x_len.to(DEVICE), x_bar.to(DEVICE), x_col.to(DEVICE)
            loss, bar_loss, col_loss, mse, acc = model.loss_function(d.float(), x.float(), y.float(),x_len.float(),x_bar.float(),x_col.float())
            test_loss += loss
            epoch_bar_loss += bar_loss
            epoch_col_loss += col_loss
            mse_bar += mse
            acc_bar += acc
    test_loss /= len(test_loader.dataset)
    epoch_bar_loss /= len(test_loader.dataset)
    epoch_col_loss /= len(test_loader.dataset)
    acc_bar /= len(test_loader.dataset)
    mse_bar /= len(test_loader.dataset)
    
    return test_loss, epoch_bar_loss, epoch_col_loss, mse_bar, acc_bar
  

In [45]:
def train(args, train_loader, test_loader, diva, optimizer, end_epoch, start_epoch=0, save_folder='sd_1.0.0',save_interval=5):
    
    epoch_loss_sup = []
    test_loss = []
    
    for epoch in range(start_epoch+1, end_epoch+1):
        diva.beta = min([args.beta, args.beta * (epoch - args.prewarmup * 1.) / (args.warmup)])
        if epoch< args.prewarmup:
            diva.beta = args.beta/args.prewarmup
        train_loss, avg_loss_bar, avg_loss_col, mtr, atr = train_single_epoch(train_loader, diva, optimizer, epoch)
        str_loss_sup = train_loss
        epoch_loss_sup.append(train_loss)
        str_print = "epoch {}: avg train loss {:.2f}".format(epoch, str_loss_sup)
        str_print += ", bar train loss {:.3f}".format(avg_loss_bar)
        str_print += ", col train loss {:.3f}".format(avg_loss_col)
        print(str_print)

        rec_loss_train = diva.rec_beta * avg_loss_bar + diva.rec_gamma * avg_loss_col
        dis_loss_train = train_loss - rec_loss_train

        test_lss, avg_loss_bar_test, avg_loss_col_test, mte, ate = test_single_epoch(test_loader, diva, epoch)
        test_loss.append(test_lss)
       
        str_print = "epoch {}: avg test  loss {:.2f}".format(epoch, test_lss)
        str_print += ", bar  test loss {:.3f}".format(avg_loss_bar_test)
        str_print += ", col  test loss {:.3f}".format(avg_loss_col_test)
        print(str_print)

        rec_loss_test = diva.rec_beta * avg_loss_bar_test + diva.rec_gamma * avg_loss_col_test
        dis_loss_test = test_lss - rec_loss_test

        if writer is not None:
            
            writer.add_scalars("Total_Loss", {'train': train_loss, 'test': test_lss} ,epoch)
            writer.add_scalars("Reconstruction_vs_Disentanglement",{'rec':rec_loss_train, 'dis':dis_loss_train}, epoch)
            writer.add_scalars("bar_mse",{'train': mtr, 'test':mte}, epoch)
            writer.add_scalars("bar_acc",{'train': atr, 'test':ate}, epoch)

        if epoch % save_interval == 0:
            save_reconstructions(epoch, test_loader, diva, name=save_folder)
            save_reconstructions(epoch, train_loader, diva, name=save_folder, estr='tr')
        
        
        if epoch % 50 == 0:
            torch.save(diva.state_dict(), f'{link}/saved_models/{save_folder}/checkpoints/{epoch}.pth')

    if writer is not None:
        writer.flush()

    epoch_loss_sup = [i.detach().cpu().numpy() for i in epoch_loss_sup]
    test_loss = [i.detach().cpu().numpy() for i in test_loss]
    return epoch_loss_sup, test_loss

In [59]:
def save_reconstructions(epoch, test_loader, diva, name='diva', estr=''):
    a = next(enumerate(test_loader))
    with torch.no_grad():
        diva.eval()
        d = a[1][2][:10].to(DEVICE).float()
        x = a[1][0][:10].to(DEVICE).float()
        x_org = a[1][-1][:10].to(DEVICE).float()
        x_col = a[1][4][:10].to(DEVICE).float()
        y = a[1][1][:10].to(DEVICE).float()
        m = a[1][-2][:10].to(DEVICE).float()
        x_2, x_2var, x_3 ,qz, pz, z_q = diva(d,x,y,x_col)
        out = diva.px.reconstruct_image(x_2, x_2var, x_3)

    plt.figure(figsize=(80,20))
    fig, ax = plt.subplots(nrows=10, ncols=2)

    ax[0,0].set_title("Original")
    ax[0,1].set_title("Reconstructed")

    for i in range(10):
        ax[i, 1].imshow(out[i])
        ax[i, 0].imshow(x_org[i].cpu().permute(1,2,0))
        ax[i, 0].xaxis.set_visible(False)
        ax[i, 0].yaxis.set_visible(False)
        ax[i, 1].xaxis.set_visible(False)
        ax[i, 1].yaxis.set_visible(False)
    fig.tight_layout(pad=0.1)
    plt.savefig(f'{link}/saved_models/{name}/reconstructions/e{epoch}{estr}.png')
    plt.close('all')

In [47]:
DEVICE

device(type='cuda')

## Model Training

In [50]:
default_args = diva_args(z_dim=512, rec_alpha = 1, rec_beta = 1, rec_gamma = 1, 
                         beta=1, warmup=1, prewarmup=0)

In [51]:
diva = StampDIVA(default_args).to(DEVICE)

In [52]:
#diva.load_state_dict(torch.load(f'{link}/saved_models/new/NVAE/checkpoints/193.pth'))

In [53]:
train_loader = DataLoader(RNA_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(RNA_dataset_test, batch_size=128)

In [54]:
#optimizer = optim.SGD(diva.parameters(), lr=0.00001, momentum=0.1, nesterov=True)
#optimizer = optim.Adam(diva.parameters(), lr=0.0005)
optimizer = optim.RMSprop(diva.parameters(), lr=0.001, eps=0.1, momentum=0.1)

In [55]:
RNA_dataset.x_len.min(), RNA_dataset.x_len.max()

(10, 100)

In [56]:
writer.flush()

In [61]:
%tensorboard --logdir="D:/users/Marko/downloads/mirna/saved_models/new/IVAE3/tensorboard/"

In [60]:
lss, lss_t = train(default_args, train_loader, test_loader, diva, optimizer, 500, 5, save_folder="new/NVAE5",save_interval=5)

Epoch 6: 272batch [00:32,  8.47batch/s, loss=296]


epoch 6: avg train loss 289.28, bar train loss 7.897, col train loss 281.069


Epoch 7: 1batch [00:00,  8.77batch/s, loss=292]

epoch 6: avg test  loss 289.21, bar  test loss 7.874, col  test loss 281.057


Epoch 7: 272batch [00:32,  8.47batch/s, loss=290]


...

epoch 72: avg train loss 288.95, bar train loss 7.840, col train loss 281.047


Epoch 73: 1batch [00:00,  8.00batch/s, loss=289]

epoch 72: avg test  loss 288.93, bar  test loss 7.839, col  test loss 281.030


Epoch 73: 8batch [00:01,  7.40batch/s, loss=289]


KeyboardInterrupt: 

In [35]:
#torch.save(diva.state_dict(), f'{link}/saved_models/new/NVAE/checkpoints/193.pth')

In [36]:
lss2, lss_t2 = train(default_args, train_loader, test_loader, diva, optimizer, 1000, 193, save_folder="new/NVAE1")

Epoch 194: 272batch [01:15,  3.58batch/s, loss=2.74e+3]


epoch 194: avg train loss 2747.05, bar train loss 1.683, col train loss 267.919
epoch 194: avg test  loss 2787.57, bar  test loss 1.810, col  test loss 271.655


Epoch 195: 272batch [01:07,  4.00batch/s, loss=2.71e+3]


epoch 195: avg train loss 2746.20, bar train loss 1.674, col train loss 267.840
epoch 195: avg test  loss 2787.51, bar  test loss 1.822, col  test loss 271.740


Epoch 196: 272batch [01:08,  4.00batch/s, loss=2.72e+3]


...

epoch 999: avg train loss 2729.16, bar train loss 1.613, col train loss 265.966


Epoch 1000: 0batch [00:00, ?batch/s]

epoch 999: avg test  loss 2790.70, bar  test loss 1.794, col  test loss 271.837


Epoch 1000: 272batch [01:19,  3.43batch/s, loss=2.75e+3]


epoch 1000: avg train loss 2729.17, bar train loss 1.614, col train loss 265.974
epoch 1000: avg test  loss 2791.17, bar  test loss 1.803, col  test loss 271.833


In [37]:
lss, lss_t = train(default_args, train_loader, test_loader, diva, optimizer, 3000, 1000, save_folder="new/NVAE1")

Epoch 1001: 272batch [01:06,  4.06batch/s, loss=2.77e+3]


epoch 1001: avg train loss 2729.33, bar train loss 1.612, col train loss 265.977


Epoch 1002: 0batch [00:00, ?batch/s]

epoch 1001: avg test  loss 2791.74, bar  test loss 1.812, col  test loss 271.827


Epoch 1002: 272batch [01:06,  4.09batch/s, loss=2.72e+3]


...

Epoch 2407: 0batch [00:00, ?batch/s]

epoch 2406: avg test  loss 2794.91, bar  test loss 1.912, col  test loss 272.160


Epoch 2407: 272batch [01:27,  3.10batch/s, loss=2.7e+3] 


epoch 2407: avg train loss 2748.50, bar train loss 1.780, col train loss 267.738


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 7680000 bytes.

In [ ]:
def plot_loss_acc(lss, lss_t):
    fig,ax = plt.subplots()
    ax.plot(lss, label="train loss")
    ax.plot(lss_t, label = "test loss")
    #ax1 = ax.twinx()
    #ax1.plot(yacc, label = "train accuracy", ls='--')
    #ax1.plot(yacc_t, label = "test accuracy", ls='--')

    lines, labels = ax.get_legend_handles_labels()
    #lines2, labels2 = ax1.get_legend_handles_labels()

    ax.legend(lines, labels)

In [ ]:
plot_loss_acc(lss, lss_t)

In [ ]:
plot_loss_acc(lss3, lss_t3, yacc3, yacc_t3)

In [ ]:
def plot_change_latent_var(diva, lat_space="y", var_idx=[0,1,2,3,4,5,6,7], step = 5):
    a = next(enumerate(test_loader))
    with torch.no_grad():
        diva.eval()
        d = a[1][2][:len(var_idx)].to(DEVICE).float()
        x = a[1][0][:len(var_idx)].to(DEVICE).float()
        y = a[1][1][:len(var_idx)].to(DEVICE).float()

        zx, zx_sc = diva.qzx(x)
        zy, zy_sc = diva.qzy(x)
        zd, zd_sc =  diva.qzd(x)

        print(torch.max(zy), torch.min(zy), "sdmax:", torch.max(zy_sc))

        out = change(zx, zy, zd, var_idx, lat_space, diva, step)
    
    fig, ax = plt.subplots(ncols=out.shape[0],nrows=len(var_idx),figsize=(10*4*out.shape[0],10*len(var_idx)))
    for i in range(out.shape[0]):
      for j in range(len(var_idx)):
        ax[j,i].imshow(out[i,j])

In [ ]:
def change(zx, zy, zd, idx, lat = "y", model=diva, step = 2):
    
    dif = np.arange(-30,15,step)
    print(torch.max(zy), torch.min(zy))
    out = np.zeros((dif.shape[0], len(idx), 25, 100 ,3))  
    #print(zy.shape, dif.shape[0])
    for i in range(dif.shape[0]):
      for j in range(len(idx)):
        if lat == "y":
            zy[j,idx] = dif[i]
        elif lat == "x":
            zx[j,idx] = dif[i]
        elif lat == "d":
            zd[j,idx] = dif[i]
        len_, bar, col = model.px(zd[j],zx[j],zy[j])
        out[i,j] = model.px.reconstruct_image(len_[None,:], bar, col)
    
    return out



In [ ]:
plot_change_latent_var(diva)

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(50,120), [i.cpu().detach().numpy() for i in lss2], label="train loss")
ax.plot(np.arange(50,120), [i.cpu().detach().numpy() for i in lss_t2], label = "testloss")
ax1 = ax.twinx()
ax1.plot(np.arange(50,120), yacc2, label = "train")
ax1.plot(np.arange(50,120), yacc_t2, label = "test")

plt.legend()

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(120,180), [i.cpu().detach().numpy() for i in lss3], label="train loss")
ax.plot(np.arange(120,180), [i.cpu().detach().numpy() for i in lss_t3], label = "testloss")
ax1 = ax.twinx()
ax1.plot(np.arange(120,180), yacc3, label = "train",c='green')
ax1.plot(np.arange(120,180), yacc_t3, label = "test")

plt.legend()

# Model Evaluation

## Sampling from trained model

In [ ]:
def plot_latent_space(lat_space="y"):
    '''
    lat_space: y, d, x
    '''

    

In [ ]:
plot(x, out, 0)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3)
for i in range(9):
  ax[i//3, i%3].imshow(x[i].cpu().permute(1,2,0))
  
plt.savefig('divastamporg.png')